In [33]:
from typing import Dict, List, Optional
import nest_asyncio
from pydantic import BaseModel, Field
from pydantic_ai import Agent, ModelRetry, RunContext, Tool
from pydantic_ai.models.gemini import GeminiModel
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GEMINI_API_KEY"] = os.getenv("GEMINI_API_KEY")
nest_asyncio.apply()


model = GeminiModel('gemini-1.5-pro')


In [8]:
# --------------------------------------------------------------
# 1. Simple Agent - Hello World Example
# --------------------------------------------------------------
"""
This example demonstrates the basic usage of PydanticAI agents.
Key concepts:
- Creating a basic agent with a system prompt
- Running synchronous queries
- Accessing response data, message history, and costs
"""
from rich import print

agent1 = Agent(
    model=model,
    system_prompt="You are a helpful customer support agent. Be concise and friendly.",
)

# Example usage of basic agent
response = agent1.run_sync("How can I track my order #12345?")
print(response.data)
print(response.all_messages())


response2 = agent1.run_sync(
    user_prompt="What was my previous question?",
    message_history=response.new_messages(),
)
print(response2.data)

Hi there!  You can track your order #12345 here: .  Let me know if you have any other questions.

[
    ModelRequest(
        parts=[
            SystemPromptPart(
                content='You are a helpful customer support agent. Be concise and friendly.',
                dynamic_ref=None,
                part_kind='system-prompt'
            ),
            UserPromptPart(
                content='How can I track my order #12345?',
                timestamp=datetime.datetime(2025, 1, 16, 14, 28, 11, 120566, tzinfo=datetime.timezone.utc),
                part_kind='user-prompt'
            )
        ],
        kind='request'
    ),
    ModelResponse(
        parts=[
            TextPart(
                content='Hi there!  You can track your order #12345 here: [insert tracking link here].  Let me know
if you have any other questions.\n',
                part_kind='text'
            )
        ],
        timestamp=datetime.datetime(2025, 1, 16, 14, 28, 13, 249145, tzinfo=datetime.timezone.utc),
        kind='response'
    )
]

Your previous question was how to track your order #12345.

In [11]:
response2.new_messages()

[ModelRequest(parts=[UserPromptPart(content='What was my previous question?', timestamp=datetime.datetime(2025, 1, 16, 14, 28, 13, 384107, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'),
 ModelResponse(parts=[TextPart(content='Your previous question was how to track your order #12345.\n', part_kind='text')], timestamp=datetime.datetime(2025, 1, 16, 14, 28, 13, 944820, tzinfo=datetime.timezone.utc), kind='response')]

In [21]:
# --------------------------------------------------------------
# 2. Agent with Structured Response
# --------------------------------------------------------------
"""
This example shows how to get structured, type-safe responses from the agent.
Key concepts:
- Using Pydantic models to define response structure
- Type validation and safety
- Field descriptions for better model understanding
"""

class ResponseModel(BaseModel):
    """Structured response with metadata."""

    response: str
    needs_escalation: bool
    follow_up_required: bool
    sentiment: str = Field(description="Customer sentiment analysis")


agent2 = Agent(
    model=model,
    result_type=ResponseModel,
    system_prompt=(
        "You are an intelligent customer support agent. "
        "Analyze queries carefully and provide structured responses."
    ),
)

response = agent2.run_sync("How can I track my order #12345?")
print(response.data.model_dump_json(indent=2))


{
  "response": "I can help you track your order.  However, to do so I'll need your email address or phone number 
associated with the order.",
  "needs_escalation": false,
  "follow_up_required": true,
  "sentiment": "neutral"
}

In [22]:
response2 = agent2.run_sync(
    user_prompt="I am not satisfied with the service",
    message_history=response.new_messages(),
)
print(response2.data)

ResponseModel(
    response='I apologize that I could not help you track your order.  I will escalate this issue to a human agent 
who can assist you.',
    needs_escalation=True,
    follow_up_required=True,
    sentiment='negative'
)

In [23]:
from pydantic import BaseModel


def to_markdown(data, indent=0):
    markdown = ""
    if isinstance(data, BaseModel):
        data = data.model_dump()
    if isinstance(data, dict):
        for key, value in data.items():
            markdown += f"{'#' * (indent + 2)} {key.upper()}\n"
            if isinstance(value, (dict, list, BaseModel)):
                markdown += to_markdown(value, indent + 1)
            else:
                markdown += f"{value}\n\n"
    elif isinstance(data, list):
        for item in data:
            if isinstance(item, (dict, list, BaseModel)):
                markdown += to_markdown(item, indent)
            else:
                markdown += f"- {item}\n"
        markdown += "\n"
    else:
        markdown += f"{data}\n\n"
    return markdown

In [24]:
# --------------------------------------------------------------
# 3. Agent with Structured Response & Dependencies
# --------------------------------------------------------------
"""
This example demonstrates how to use dependencies and context in agents.
Key concepts:
- Defining complex data models with Pydantic
- Injecting runtime dependencies
- Using dynamic system prompts
"""


# Define order schema
class Order(BaseModel):
    """Structure for order details."""

    order_id: str
    status: str
    items: List[str]


# Define customer schema
class CustomerDetails(BaseModel):
    """Structure for incoming customer queries."""

    customer_id: str
    name: str
    email: str
    orders: Optional[List[Order]] = None


# Agent with structured output and dependencies
agent5 = Agent(
    model=model,
    result_type=ResponseModel,
    deps_type=CustomerDetails,
    retries=3,
    system_prompt=(
        "You are an intelligent customer support agent. "
        "Analyze queries carefully and provide structured responses. "
        "Always greet the customer and provide a helpful response."
    ),  # These are known when writing the code
)


# Add dynamic system prompt based on dependencies
@agent5.system_prompt
async def add_customer_name(ctx: RunContext[CustomerDetails]) -> str:
    return f"Customer details: {to_markdown(ctx.deps)}"  # These depend in some way on context that isn't known until runtime


customer = CustomerDetails(
    customer_id="1",
    name="John Doe",
    email="john.doe@example.com",
    orders=[
        Order(order_id="12345", status="shipped", items=["Blue Jeans", "T-Shirt"]),
    ],
)

response = agent5.run_sync(user_prompt="What did I order?", deps=customer)

response.all_messages()
print(response.data.model_dump_json(indent=2))

print(
    "Customer Details:\n"
    f"Name: {customer.name}\n"
    f"Email: {customer.email}\n\n"
    "Response Details:\n"
    f"{response.data.response}\n\n"
    "Status:\n"
    f"Follow-up Required: {response.data.follow_up_required}\n"
    f"Needs Escalation: {response.data.needs_escalation}"
)


{
  "response": "Hello John Doe, you ordered a Blue Jeans and a T-Shirt. Your order (12345) has been shipped.",
  "needs_escalation": false,
  "follow_up_required": false,
  "sentiment": "positive"
}

Customer Details:
Name: John Doe
Email: john.doe@example.com

Response Details:
Hello John Doe, you ordered a Blue Jeans and a T-Shirt. Your order (12345) has been shipped.

Status:
Follow-up Required: False
Needs Escalation: False

In [25]:
response = agent5.run_sync(user_prompt="thanks")

In [27]:
print(response.data)

ResponseModel(
    response="You're welcome! Is there anything else I can help you with?",
    needs_escalation=False,
    follow_up_required=False,
    sentiment='positive'
)

In [28]:
# --------------------------------------------------------------
# 4. Agent with Tools
# --------------------------------------------------------------

"""
This example shows how to enhance agents with custom tools.
Key concepts:
- Creating and registering tools
- Accessing context in tools
"""

shipping_info_db: Dict[str, str] = {
    "12345": "Shipped on 2024-12-01",
    "67890": "Out for delivery",
}


def get_shipping_info(ctx: RunContext[CustomerDetails]) -> str:
    """Get the customer's shipping information."""
    return shipping_info_db[ctx.deps.orders[0].order_id]


# Agent with structured output and dependencies
agent5 = Agent(
    model=model,
    result_type=ResponseModel,
    deps_type=CustomerDetails,
    retries=3,
    system_prompt=(
        "You are an intelligent customer support agent. "
        "Analyze queries carefully and provide structured responses. "
        "Use tools to look up relevant information."
        "Always greet the customer and provide a helpful response."
    ),  # These are known when writing the code
    tools=[Tool(get_shipping_info, takes_ctx=True)],  # Add tool via kwarg
)


@agent5.system_prompt
async def add_customer_name(ctx: RunContext[CustomerDetails]) -> str:
    return f"Customer details: {to_markdown(ctx.deps)}"


response = agent5.run_sync(
    user_prompt="What's the status of my last order?", deps=customer
)

response.all_messages()
print(response.data.model_dump_json(indent=2))

print(
    "Customer Details:\n"
    f"Name: {customer.name}\n"
    f"Email: {customer.email}\n\n"
    "Response Details:\n"
    f"{response.data.response}\n\n"
    "Status:\n"
    f"Follow-up Required: {response.data.follow_up_required}\n"
    f"Needs Escalation: {response.data.needs_escalation}"
)



{
  "response": "Your order (12345) has shipped on 2024-12-01.",
  "needs_escalation": false,
  "follow_up_required": false,
  "sentiment": "positive"
}

Customer Details:
Name: John Doe
Email: john.doe@example.com

Response Details:
Your order (12345) has shipped on 2024-12-01.

Status:
Follow-up Required: False
Needs Escalation: False

In [36]:

# --------------------------------------------------------------
# 5. Agent with Reflection and Self-Correction
# --------------------------------------------------------------

"""
This example demonstrates advanced agent capabilities with self-correction.
Key concepts:
- Implementing self-reflection
- Handling errors gracefully with retries
- Using ModelRetry for automatic retries
- Decorator-based tool registration
"""

# Simulated database of shipping information
shipping_info_db: Dict[str, str] = {
    "#12345": "Shipped on 2024-12-01",
    "#67890": "Out for delivery",
}

customer = CustomerDetails(
    customer_id="1",
    name="John Doe",
    email="john.doe@example.com",
)

# Agent with reflection and self-correction
agent5 = Agent(
    model=model,
    result_type=ResponseModel,
    deps_type=CustomerDetails,
    retries=3,
    system_prompt=(
        "You are an intelligent customer support agent. "
        "Analyze queries carefully and provide structured responses. "
        "Use tools to look up relevant information. "
        "Always greet the customer and provide a helpful response."
    ),
)


@agent5.tool_plain()  # Add plain tool via decorator
def get_shipping_status(order_id: str) -> str:
    """Get the shipping status for a given order ID."""
    shipping_status = shipping_info_db.get(order_id)
    if shipping_status is None:
        raise ModelRetry(
            f"No shipping information found for order ID {order_id}. "
            "Make sure the order ID starts with a #: e.g, #624743 "
            "Self-correct this if needed and try again the tool"
        )
    return shipping_info_db[order_id]


# Example usage
response = agent5.run_sync(
    user_prompt="What's the status of my last order 12345?", deps=customer
)

display(response.all_messages())
print(response.data.model_dump_json(indent=2))

[ModelRequest(parts=[SystemPromptPart(content='You are an intelligent customer support agent. Analyze queries carefully and provide structured responses. Use tools to look up relevant information. Always greet the customer and provide a helpful response.', dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content="What's the status of my last order 12345?", timestamp=datetime.datetime(2025, 1, 16, 16, 1, 19, 142663, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'),
 ModelResponse(parts=[ToolCallPart(tool_name='get_shipping_status', args=ArgsDict(args_dict={'order_id': '12345'}), tool_call_id=None, part_kind='tool-call')], timestamp=datetime.datetime(2025, 1, 16, 16, 1, 21, 667406, tzinfo=datetime.timezone.utc), kind='response'),
 ModelRequest(parts=[RetryPromptPart(content='No shipping information found for order ID 12345. Make sure the order ID starts with a #: e.g, #624743 Self-correct this if needed and try again the tool', tool_name='get_shipping

{
  "response": "Your order #12345 was shipped on 2024-12-01.",
  "needs_escalation": false,
  "follow_up_required": false,
  "sentiment": "positive"
}